In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import bisect
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
session =  pd.read_csv("sessions.csv")

In [3]:
session

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0
...,...,...,...,...,...,...
10567732,9uqfg8txu3,dashboard,view,dashboard,Windows Desktop,556.0
10567733,9uqfg8txu3,edit,view,edit_profile,Windows Desktop,6624.0
10567734,9uqfg8txu3,webcam_upload,-unknown-,-unknown-,Windows Desktop,200125.0
10567735,9uqfg8txu3,active,-unknown-,-unknown-,-unknown-,17624.0


In [4]:
session.dtypes

user_id           object
action            object
action_type       object
action_detail     object
device_type       object
secs_elapsed     float64
dtype: object

In [5]:
session['id'] = session['user_id']
session = session.drop(['user_id'],axis=1)

In [6]:
(session.shape[0] - session.count())/session.shape[0]

action           0.007535
action_type      0.106570
action_detail    0.106570
device_type      0.000000
secs_elapsed     0.012872
id               0.003264
dtype: float64

In [7]:
session[['action','action_type','action_detail','device_type','id']]=session[['action','action_type','action_detail','device_type','id']].fillna('NULL')
session['secs_elapsed'] = session['secs_elapsed'].fillna(0)

In [8]:
(session.shape[0] - session.count())/session.shape[0]

action           0.0
action_type      0.0
action_detail    0.0
device_type      0.0
secs_elapsed     0.0
id               0.0
dtype: float64

In [9]:
session

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NULL,NULL,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NULL,NULL,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NULL,NULL,Windows Desktop,435.0,d1mm9tcy42
...,...,...,...,...,...,...
10567732,dashboard,view,dashboard,Windows Desktop,556.0,9uqfg8txu3
10567733,edit,view,edit_profile,Windows Desktop,6624.0,9uqfg8txu3
10567734,webcam_upload,-unknown-,-unknown-,Windows Desktop,200125.0,9uqfg8txu3
10567735,active,-unknown-,-unknown-,-unknown-,17624.0,9uqfg8txu3


In [10]:
session_by_id_time = session.groupby(['id'])['secs_elapsed'].sum().astype(int).reset_index()

In [11]:
session_by_id_time

,id,secs_elapsed
0,00023iyk9l,867896
1,0010k6l0om,586543
2,001wyh0pz8,282965
3,0028jgx1x1,297010
4,002qnbzfs5,6487080
...,...,...
135479,zzxox7jnrx,639436
135480,zzy7t0y9cm,73771
135481,zzysuoqg6x,1731
135482,zzywmcn0jv,2149949


In [12]:
session_num_of_action = session.groupby(['id'])['action'].value_counts().to_frame()
session_num_of_action

action
id         action                             
00023iyk9l show                              9
           requested                         5
           dashboard                         4
           index                             4
           personalize                       4
...                                        ...
zzzlylp57e create                            1
           lookup                            1
           message_to_host_focus             1
           other_hosting_reviews_first       1
           pending                           1

[1776035 rows x 1 columns]

In [13]:
session_id_action = session_num_of_action.stack().to_frame().reset_index()
session_id_action = session_id_action.drop(['level_2'],axis=1).rename(columns={0:'num'})
session_id_action

,id,action,num
0,00023iyk9l,show,9
1,00023iyk9l,requested,5
2,00023iyk9l,dashboard,4
3,00023iyk9l,index,4
4,00023iyk9l,personalize,4
...,...,...,...
1776030,zzzlylp57e,create,1
1776031,zzzlylp57e,lookup,1
1776032,zzzlylp57e,message_to_host_focus,1
1776033,zzzlylp57e,other_hosting_reviews_first,1


In [14]:
session_action_num = session_id_action.groupby(['action'])['num'].sum().to_frame().reset_index()
session_action_num

,action,num
0,10,3215
1,11,716
2,12,2209
3,15,1053
4,NULL,79626
...,...,...
355,weibo_signup_referral_finish,45
356,why_host,180
357,widget,75
358,wishlists,1


In [15]:
session_action_num = session.groupby(['action']).count().reset_index().rename(columns={'id':'num'})[['action','num']]
session_action_num

,action,num
0,10,3215
1,11,716
2,12,2209
3,15,1053
4,NULL,79626
...,...,...
355,weibo_signup_referral_finish,45
356,why_host,180
357,widget,75
358,wishlists,1


In [16]:
action_option = session_action_num[session_action_num['num'] >100000]['action'].to_list()
action_option

['active',
 'ajax_refresh_subtotal',
 'campaigns',
 'collections',
 'create',
 'dashboard',
 'edit',
 'header_userpic',
 'index',
 'lookup',
 'personalize',
 'reviews',
 'search',
 'search_results',
 'show',
 'similar_listings',
 'similar_listings_v2',
 'social_connections',
 'update']

In [17]:
session_id_action = session[session['action'].apply(lambda x: True if (x in action_option) else False)]
# session_id_action = session_id_action.drop(['num'],axis=1)
session_id_action

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NULL,NULL,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NULL,NULL,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NULL,NULL,Windows Desktop,435.0,d1mm9tcy42
...,...,...,...,...,...,...
10567730,edit,view,edit_profile,Windows Desktop,1752436.0,9uqfg8txu3
10567731,edit,view,edit_profile,Windows Desktop,15875.0,9uqfg8txu3
10567732,dashboard,view,dashboard,Windows Desktop,556.0,9uqfg8txu3
10567733,edit,view,edit_profile,Windows Desktop,6624.0,9uqfg8txu3


In [18]:
session_num_of_action = session.groupby(['id']).count().reset_index()
session_num_of_action

,id,action,action_type,action_detail,device_type,secs_elapsed
0,00023iyk9l,40,40,40,40,40
1,0010k6l0om,63,63,63,63,63
2,001wyh0pz8,90,90,90,90,90
3,0028jgx1x1,31,31,31,31,31
4,002qnbzfs5,789,789,789,789,789
...,...,...,...,...,...,...
135479,zzxox7jnrx,89,89,89,89,89
135480,zzy7t0y9cm,8,8,8,8,8
135481,zzysuoqg6x,3,3,3,3,3
135482,zzywmcn0jv,51,51,51,51,51


In [19]:
session_action_by_id2 = session_id_action.groupby(['id'])['action'].value_counts().unstack().fillna(0).astype(int).reset_index()

In [20]:
session_action_by_id2

action,id,active,ajax_refresh_subtotal,campaigns,collections,create,dashboard,edit,header_userpic,index,lookup,personalize,reviews,search,search_results,show,similar_listings,similar_listings_v2,social_connections,update
0,00023iyk9l,0,2,0,0,0,4,0,2,4,0,4,0,0,1,9,3,0,0,0
1,0010k6l0om,0,8,0,0,0,2,0,1,5,0,8,0,0,8,20,0,8,0,0
2,001wyh0pz8,4,0,3,4,1,0,0,0,2,0,0,0,66,0,6,0,0,2,1
3,0028jgx1x1,1,0,0,0,1,0,0,0,0,0,0,5,9,0,15,0,0,0,0
4,002qnbzfs5,29,0,28,0,21,2,0,2,115,0,0,13,125,0,232,0,0,72,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133690,zzxox7jnrx,0,10,0,0,1,3,3,1,5,1,5,0,0,12,11,0,10,0,1
133691,zzy7t0y9cm,0,1,0,0,1,1,0,1,0,0,2,0,0,0,1,0,0,0,0
133692,zzysuoqg6x,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
133693,zzywmcn0jv,0,11,0,0,1,0,0,1,1,0,1,0,0,1,16,11,0,0,0


In [21]:
session_action_by_id = session.groupby(['id'])['action'].value_counts().unstack().fillna(0).astype(int).reset_index()

In [22]:
session_action_by_id = session_action_by_id.rename(columns={'NULL':'action_NULL'})

In [23]:
session_action_by_id

action,id,10,11,12,15,action_NULL,about_us,accept_decline,account,acculynk_bin_check_failed,...,view,views,views_campaign,views_campaign_rules,webcam_upload,weibo_signup_referral_finish,why_host,widget,wishlists,zendesk_login_jwt
0,00023iyk9l,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0010k6l0om,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,001wyh0pz8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0028jgx1x1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,002qnbzfs5,9,0,0,0,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,zzxox7jnrx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135480,zzy7t0y9cm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135481,zzysuoqg6x,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135482,zzywmcn0jv,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
session_action_type_by_id = session.groupby(['id'])['action_type'].value_counts().unstack().fillna(0).astype(int).reset_index()
session_action_type_by_id = session_action_type_by_id.rename(columns={'NULL':'action_type_NULL'})
session_action_type_by_id = session_action_type_by_id.rename(columns={'-unknown-':'action_type_-unknown-'})

In [25]:
session_action_type_by_id

action_type,id,action_type_-unknown-,action_type_NULL,booking_request,booking_response,click,data,message_post,modify,partner_callback,submit,view
0,00023iyk9l,0,3,1,0,4,9,1,0,1,0,21
1,0010k6l0om,5,15,0,0,16,9,0,0,1,0,17
2,001wyh0pz8,6,5,0,0,66,2,0,0,0,3,8
3,0028jgx1x1,1,0,0,0,9,5,0,0,0,1,15
4,002qnbzfs5,184,77,1,0,140,140,16,0,0,15,216
...,...,...,...,...,...,...,...,...,...,...,...,...
135479,zzxox7jnrx,13,17,0,0,25,8,0,0,1,2,23
135480,zzy7t0y9cm,0,1,0,0,2,3,0,0,0,1,1
135481,zzysuoqg6x,1,0,0,0,0,1,0,0,0,1,0
135482,zzywmcn0jv,2,8,0,0,14,13,2,0,0,3,9


In [26]:
session_action_detail_num = session.groupby(['action_detail']).count().reset_index().rename(columns={'id':'num'})[['action_detail','num']]

In [27]:
session_action_detail_num

,action_detail,num
0,-unknown-,1031141
1,NULL,1126204
2,account_notification_settings,21802
3,account_payment_methods,1005
4,account_payout_preferences,4569
...,...,...
151,wishlist_content_update,706824
152,wishlist_note,961
153,your_listings,30980
154,your_reservations,1283


In [28]:
action_option = session_action_detail_num[session_action_detail_num['num'] >100000]['action_detail'].to_list()

In [29]:
action_option.remove('-unknown-')

In [30]:
action_option.remove('NULL')

In [31]:
action_option

['change_trip_characteristics',
 'dashboard',
 'edit_profile',
 'header_userpic',
 'listing_reviews',
 'message_thread',
 'p3',
 'similar_listings',
 'update_listing',
 'user_profile',
 'user_social_connections',
 'user_wishlists',
 'view_search_results',
 'wishlist_content_update']

In [32]:
session_id_action_detail = session[session['action_detail'].apply(lambda x: True if (x in action_option) else False)]

In [33]:
session_id_action_detail

,action,action_type,action_detail,device_type,secs_elapsed,id
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
5,search_results,click,view_search_results,Windows Desktop,7703.0,d1mm9tcy42
7,personalize,data,wishlist_content_update,Windows Desktop,831.0,d1mm9tcy42
8,index,view,view_search_results,Windows Desktop,20842.0,d1mm9tcy42
...,...,...,...,...,...,...
10567729,header_userpic,data,header_userpic,Windows Desktop,1939.0,9uqfg8txu3
10567730,edit,view,edit_profile,Windows Desktop,1752436.0,9uqfg8txu3
10567731,edit,view,edit_profile,Windows Desktop,15875.0,9uqfg8txu3
10567732,dashboard,view,dashboard,Windows Desktop,556.0,9uqfg8txu3


In [34]:
session_action_detail_by_id = session_id_action_detail.groupby(['id'])['action_detail'].value_counts().unstack().fillna(0).astype(int).reset_index()
session_action_detail_by_id

action_detail,id,change_trip_characteristics,dashboard,edit_profile,header_userpic,listing_reviews,message_thread,p3,similar_listings,update_listing,user_profile,user_social_connections,user_wishlists,view_search_results,wishlist_content_update
0,00023iyk9l,2,4,0,2,0,0,6,3,0,0,0,0,5,4
1,0010k6l0om,8,2,0,1,0,0,13,0,0,0,0,0,10,8
2,001wyh0pz8,0,0,0,0,0,0,2,0,0,2,2,4,66,0
3,0028jgx1x1,0,0,0,0,5,0,9,0,0,6,0,0,9,0
4,002qnbzfs5,0,2,0,2,10,0,25,0,0,109,72,0,125,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128766,zzxox7jnrx,10,3,3,1,0,0,3,0,0,4,0,0,16,5
128767,zzy7t0y9cm,1,1,0,1,0,0,0,0,0,0,0,0,0,2
128768,zzysuoqg6x,0,0,0,1,0,0,0,0,0,0,0,0,0,0
128769,zzywmcn0jv,11,0,0,1,0,0,8,11,0,0,0,0,2,1


In [35]:
session_action_detail_by_id = session.groupby(['id'])['action_detail'].value_counts().unstack().fillna(0).astype(int).reset_index()
session_action_detail_by_id = session_action_detail_by_id.rename(columns={'NULL':'action_detail_NULL'})
session_action_detail_by_id = session_action_detail_by_id.rename(columns={'-unknown-':'action_detail_-unknown-'})
session_action_detail_by_id

action_detail,id,action_detail_-unknown-,action_detail_NULL,account_notification_settings,account_payment_methods,account_payout_preferences,account_privacy_settings,account_transaction_history,admin_templates,airbnb_picks_wishlists,...,view_resolutions,view_search_results,view_security_checks,view_user_real_names,wishlist,wishlist_content_update,wishlist_note,your_listings,your_reservations,your_trips
0,00023iyk9l,0,3,0,0,0,0,0,0,0,...,0,5,0,0,0,4,0,0,0,2
1,0010k6l0om,5,15,0,0,0,0,0,0,0,...,0,10,0,0,0,8,0,0,0,0
2,001wyh0pz8,6,5,0,0,0,0,0,0,0,...,0,66,0,0,0,0,0,0,0,0
3,0028jgx1x1,1,0,0,0,0,0,0,0,0,...,0,9,0,0,0,0,0,0,0,0
4,002qnbzfs5,184,77,0,0,0,0,0,0,0,...,0,125,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,zzxox7jnrx,13,17,0,0,0,0,0,0,0,...,0,16,0,0,0,5,0,0,0,2
135480,zzy7t0y9cm,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
135481,zzysuoqg6x,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135482,zzywmcn0jv,2,8,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0


In [36]:
session_device_type_by_id = session.groupby(['id'])['device_type'].value_counts().unstack().fillna(0).astype(int).reset_index()
session_device_type_by_id = session_device_type_by_id.rename(columns={'-unknown-':'device_type_-unknown-'})

In [37]:
session_device_type_by_id

device_type,id,device_type_-unknown-,Android App Unknown Phone/Tablet,Android Phone,Blackberry,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
0,00023iyk9l,0,0,0,0,0,0,36,0,0,0,0,0,4,0
1,0010k6l0om,0,0,0,0,0,0,63,0,0,0,0,0,0,0
2,001wyh0pz8,0,90,0,0,0,0,0,0,0,0,0,0,0,0
3,0028jgx1x1,30,0,1,0,0,0,0,0,0,0,0,0,0,0
4,002qnbzfs5,14,0,0,0,0,0,0,0,0,0,0,0,775,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,zzxox7jnrx,0,0,0,0,0,0,0,0,0,89,0,0,0,0
135480,zzy7t0y9cm,0,0,0,0,0,0,0,0,0,8,0,0,0,0
135481,zzysuoqg6x,0,0,0,0,0,0,0,0,0,3,0,0,0,0
135482,zzywmcn0jv,0,0,0,0,0,0,0,0,0,51,0,0,0,0


In [38]:
results = pd.merge(session_by_id_time, session_action_by_id, how='left', on=['id'])
results = pd.merge(results, session_action_type_by_id, how='left', on=['id'])
results = pd.merge(results, session_action_detail_by_id, how='left', on=['id'])
results = pd.merge(results, session_device_type_by_id, how='left', on=['id'])

In [39]:
results

,id,secs_elapsed,10,11,12,15,action_NULL,about_us,accept_decline,account,...,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
0,00023iyk9l,867896,0,0,0,0,1,0,0,0,...,0,0,36,0,0,0,0,0,4,0
1,0010k6l0om,586543,0,0,0,0,0,0,0,0,...,0,0,63,0,0,0,0,0,0,0
2,001wyh0pz8,282965,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0028jgx1x1,297010,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,002qnbzfs5,6487080,9,0,0,0,7,0,0,0,...,0,0,0,0,0,0,0,0,775,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,zzxox7jnrx,639436,0,0,0,0,0,0,0,0,...,0,0,0,0,0,89,0,0,0,0
135480,zzy7t0y9cm,73771,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,0,0,0
135481,zzysuoqg6x,1731,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
135482,zzywmcn0jv,2149949,0,0,0,0,2,0,0,0,...,0,0,0,0,0,51,0,0,0,0
